In [6]:
!nvidia-smi

Sun Mar 27 02:36:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
%cp ../input/cs6910a2v1 ./ -r

In [8]:
ls

__notebook_source__.ipynb  cs6910a2v1/  inaturalist_12K/  nature_12K.zip


In [9]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://wandb.ai/" "https://storage.googleapis.com/wandb_datasets/nature_12K.zip" -c -O 'nature_12K.zip'

--2022-03-27 02:36:49--  https://storage.googleapis.com/wandb_datasets/nature_12K.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 108.177.13.128, 172.253.123.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.



In [10]:
!unzip "nature_12K.zip"

Archive:  nature_12K.zip
replace inaturalist_12K/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [11]:
!cp cs6910a2v1/* ./ -r

In [12]:
# from torchvision import models

In [13]:
# !conda install pytorch torchvision cudatoolkit=11.3 -c pytorch --force -y
# !pip3 install torchvision

In [14]:
# !pip3 install torch==1.4.0+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html

In [15]:
# !pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html

In [16]:
import wandb
import torch
torch.manual_seed(7)
from torch import  nn
import torch.nn.functional as F

from tqdm import tqdm 

from model import load_img_shape, load_model, load_optimizer
from data import iNaturalist, create_data

In [17]:
import wandb
from wandb.keras import WandbCallback

import cv2
import glob
import torch
import torchvision
from tqdm import tqdm 
from pandas.core.common import flatten
torch.manual_seed(7)
from torch import  nn,optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader,Dataset
from PIL import Image


def evaluate(device, loader, model, model_name):
    ''' Function to calculate accuracy to see performance of our model '''
    correct_samples = 0
    total_samples = 0
    model.eval()

    loss = .0
    criterion = nn.CrossEntropyLoss()

    with torch.no_grad():
        for x, y in tqdm(loader, total=len(loader)):
            x = x.to(device=device)
            y = y.to(device=device)

            outputs = model(x)
            outputs = F.softmax(outputs, dim=1)

            loss += criterion(outputs, y).item()

            _, predictions = outputs.max(1)
            correct_samples += (predictions == y).sum().item()
            total_samples += predictions.size(0)
           
    acc = round((correct_samples / total_samples) * 100, 4)
    return acc, loss/len(loader)     

In [18]:
def train(args):

    train_data_path = 'inaturalist_12K/train/'
    test_data_path = 'inaturalist_12K/val/'
    
    wandb.run.name = 'model-'+str(args.model)+'-epochs-'+str(args.epochs)+'-b_s-'+str(args.batch_size)+'-opt-'+str(args.optimizer) \
                      +'-lr-'+ str(args.learning_rate)+ '-da-'+str(args.data_aug)+'-pretrained-'+str(args.pretrained)

    #function to find compatible input image shape based on model
    image_shape= load_img_shape(args.model)

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    #CNN Model creation
    model = load_model(args.model, args.pretrained, args.freeze_k).to(device)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = load_optimizer(model.parameters(), args.optimizer , args.learning_rate)

    # Train Network
    for epoch in range(args.epochs):
        model.train()
        train_loader, valid_loader = create_data("train",
              train_data_path, args.data_aug, image_shape[2:], args.batch_size)
        test_loader =  create_data("test",
              test_data_path, False, image_shape[2:], args.batch_size)
      
        train_correct, train_loss = .0, .0
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            data = data.to(device=device)
            targets = targets.to(device=device)

            # forward
            outputs = model(data)
            if(args.model == "inceptionv3"):
              outputs, _ = outputs #getting final output of inceptionv3 model
              outputs = F.softmax(outputs, dim=1)
            else:
              outputs = F.softmax(outputs, dim=1)

            #computing accuracy
            _, predictions = outputs.max(1)
            train_correct += (predictions == targets).sum()

            #computing loss  
            loss = criterion(outputs, targets)
            train_loss += loss.item()

            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent step
            optimizer.step()

        train_loss /= len(train_loader)

        train_acc = round((train_correct / len(train_loader.dataset)).item() * 100, 4)
        val_acc, val_loss = evaluate(device, valid_loader, model, args.model)
        test_acc, test_loss = evaluate(device, test_loader, model, args.model)
        wandb.log(
          {'train_acc': train_acc, 'val_acc': val_acc, 'test_acc': test_acc, 'train_loss': train_loss, 'val_loss': val_loss, 'test_loss': test_loss}
        )
        print('\nEpoch ', epoch, 'train_acc', train_acc, 'val_acc', val_acc, 'test_acc', test_acc, 'train_loss', train_loss, 'val_loss', val_loss, 'test_loss', test_loss) 

In [19]:
# class dotdict(dict):
#     """dot.notation access to dictionary attributes"""
#     __getattr__ = dict.get
#     __setattr__ = dict.__setitem__
#     __delattr__ = dict.__delitem__


# cfg = {'batch_size': 64, 'data_aug': True, 'epochs': 5, 'learning_rate': 0.01, 'model': 'resnet50', 'optimizer': 'adam', 'pretrained': True, 'freeze_k': 50}
# cfg = dotdict(cfg)
# train(cfg)

In [ ]:
wandb.login()
sweep_config = {
    "name" : "cs6910_assignment2_PartB_trial_sweep_freeze_k",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "epochs" : {
          "values" : [5, 10]  
        },
        "model" : {
            "values" : ["inceptionv3", "resnet50"]
        },
        "batch_size": {
            "values": [128]
        },
        "optimizer":{
            "values": ["adam", "rmsprop"]
        },
        "learning_rate":{
            "values": [1e-3, 1e-2]
        },
        "data_aug":{
            "values": [True]
        },
        "pretrained":{
            "values": [True, False]
        },
        "freeze_k": {
            "values": [45]       
        }  
    }
}
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment2_PartB_sweep_freeze_k", entity="cs21s006_cs21s043")
print('sweep_id: ', sweep_id)

In [ ]:
def spawn_fn():
    with wandb.init(project="cs6910_assignment2_PartB_trial", entity="cs21s006_cs21s043") as run:
        config = wandb.config
        print(config)
        train(config)

In [ ]:
wandb.agent("w0lyj8a5", function=spawn_fn, count=5)

In [ ]:
wandb.login()
sweep_config = {
    "name" : "cs6910_assignment2_PartB_trial_sweep_freeze_k_inceptionresnetv2",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "epochs" : {
          "values" : [10]  
        },
        "model" : {
            "values" : ["inceptionresnetv2"]
        },
        "batch_size": {
            "values": [64]
        },
        "optimizer":{
            "values": ["adam"]
        },
        "learning_rate":{
            "values": [1e-3]
        },
        "data_aug":{
            "values": [True]
        },
        "pretrained":{
            "values": [True, False]
        },
        "freeze_k": {
            "values": [160, 163]       
        }  
    }
}
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment2_PartB_sweep_freeze_k", entity="cs21s006_cs21s043")
print('sweep_id: ', sweep_id)

def spawn_fn():
    with wandb.init(project="cs6910_assignment2_PartB_sweep_freeze_k", entity="cs21s006_cs21s043") as run:
        config = wandb.config
        print(config)
        train(config)

wandb.agent(sweep_id, function=spawn_fn, count=4)

Epoch 6

In [20]:
wandb.login()
sweep_config = {
    "name" : "cs6910_assignment2_PartB_trial_sweep_freeze_k_xception",
    "method" : "bayes",
    'metric': {
        'name': 'val_acc',
        'goal': 'maximize'
    },
    "parameters" : {
        "epochs" : {
          "values" : [10]  
        },
        "model" : {
            "values" : ["xception"]
        },
        "batch_size": {
            "values": [64]
        },
        "optimizer":{
            "values": ["adam"]
        },
        "learning_rate":{
            "values": [1e-3]
        },
        "data_aug":{
            "values": [True]
        },
        "pretrained":{
            "values": [True, False]
        },
        "freeze_k": {
            "values": [68, 70]       
        }  
    }
}
sweep_id = wandb.sweep(sweep_config, project="cs6910_assignment2_PartB_sweep_freeze_k", entity="cs21s006_cs21s043")
print('sweep_id: ', sweep_id)

def spawn_fn():
    with wandb.init(project="cs6910_assignment2_PartB_sweep_freeze_k", entity="cs21s006_cs21s043") as run:
        config = wandb.config
        print(config)
        train(config)

wandb.agent(sweep_id, function=spawn_fn, count=4)

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: wq51j29j
Sweep URL: https://wandb.ai/cs21s006_cs21s043/cs6910_assignment2_PartB_sweep_freeze_k/sweeps/wq51j29j
sweep_id:  wq51j29j


wandb: Agent Starting Run: eez2j4wm with config:
wandb: 	batch_size: 64
wandb: 	data_aug: True
wandb: 	epochs: 10
wandb: 	freeze_k: 70
wandb: 	learning_rate: 0.001
wandb: 	model: xception
wandb: 	optimizer: adam
wandb: 	pretrained: False
wandb: Currently logged in as: cs21s006_cs21s043 (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


...

100%|██████████| 32/32 [00:42<00:00,  1.34s/it]



Epoch  8 train_acc 74.3749 val_acc 76.5 test_acc 73.4 train_loss 1.7172081910126598 val_loss 1.6922727972269058 test_loss 1.727897148579359


100%|██████████| 32/32 [00:42<00:00,  1.33s/it]



Epoch  9 train_acc 75.9084 val_acc 78.2 test_acc 74.65 train_loss 1.70156943628974 val_loss 1.6746920347213745 test_loss 1.7140879295766354



test_acc,▁▁▃▅▆▆▆▇▇█
test_loss,██▆▄▃▃▃▂▂▁
train_acc,▁▂▄▅▅▆▇▇▇█
train_loss,█▆▅▄▃▃▂▂▂▁
val_acc,▁▃▃▅▆▇▆▆▇█
val_loss,█▇▆▄▃▂▃▃▂▁
test_acc,74.65
test_loss,1.71409
train_acc,75.9084
train_loss,1.70157
val_acc,78.2
